In [23]:
import requests
import csv
import os
import json

In [24]:
with open("./sonar_token.json", 'r') as f:
    api_keys = json.loads(f.read())

In [25]:
# === CONFIG ===
SONARCLOUD_TOKEN = api_keys["SONARCLOUD"]  
SONAR_PROJECT_KEY = "AsiaSzych_Tree_of_Life"  
OUTPUT_SUMMARY_FILE = "sonarcloud_metrics.csv"
OUTPUT_DETAILS_DIR = "."
BASE_URL = "https://sonarcloud.io/api"

In [53]:
METRICS = [
    "bugs",
    "vulnerabilities",
    "code_smells",
    "coverage",
    "duplicated_lines_density",
    "ncloc",  # lines of code
    "sqale_index",            # debt in minutes
    "sqale_debt_ratio",       # % debt
    "security_hotspots"
]

In [27]:
def get_all_branches(project_key):
    url = f"{BASE_URL}/project_branches/list?project={project_key}"
    response = requests.get(url, auth=(SONARCLOUD_TOKEN, ""))
    response.raise_for_status()
    branches = response.json()["branches"]
    return [b["name"] for b in branches]

In [28]:
def get_metrics(project_key, branch):
    url = f"{BASE_URL}/measures/component"
    params = {
        "component": project_key,
        "metricKeys": ",".join(METRICS),
        "branch": branch
    }
    response = requests.get(url, auth=(SONARCLOUD_TOKEN, ""), params=params)
    response.raise_for_status()
    measures = response.json().get("component", {}).get("measures", [])
    return {m["metric"]: m["value"] for m in measures}

In [29]:
def get_issues(project_key, branch):
    all_issues = []
    page = 1
    page_size = 50  

    while True:
        url = f"{BASE_URL}/issues/search"
        params = {
            "componentKeys": project_key,
            "branch": branch,
            "ps": page_size,
            "p": page
        }
        response = requests.get(url, auth=(SONARCLOUD_TOKEN, ""), params=params)
        response.raise_for_status()
        result = response.json()

        issues = result.get("issues", [])
        all_issues.extend(issues)

        if page * page_size >= result.get("total", 0):
            break
        page += 1

    return all_issues

In [30]:
def export_summary_csv(project_key, branches):
    with open(OUTPUT_SUMMARY_FILE, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["branch"] + METRICS)
        writer.writeheader()

        for branch in branches:
            print(f"Exporting summary metrics for branch: {branch}")
            data = get_metrics(project_key, branch)
            row = {"branch": branch}
            row.update(data)
            writer.writerow(row)

In [31]:
def export_detailed_issues(project_key, branches):
    os.makedirs(OUTPUT_DETAILS_DIR, exist_ok=True)

    for branch in branches:
        print(f"Exporting detailed issues for branch: {branch}")
        issues = get_issues(project_key, branch)

        processed = [
            {
                "key": i.get("key"),
                "type": i.get("type"),
                "severity": i.get("severity"),
                "component": i.get("component"),
                "line": i.get("line"),
                "message": i.get("message"),
                "status": i.get("status"),
                "creationDate": i.get("creationDate"),
                "updateDate": i.get("updateDate"),
                "rule": i.get("rule"),
            }
            for i in issues
        ]

        filename = f"{branch}_issues.json".replace("/", "_")
        filepath = os.path.join(OUTPUT_DETAILS_DIR, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(processed, f, indent=2)

In [32]:
def export_issue_severity_breakdown(project_key, branches, output_csv="issue_severity_summary.csv"):
    rows = []
    for branch in branches:
        print(f"Gathering severity breakdown for branch: {branch}")
        page = 1
        page_size = 500
        total_issues = []

        while True:
            url = f"{BASE_URL}/issues/search"
            params = {
                "componentKeys": project_key,
                "branch": branch,
                "ps": page_size,
                "p": page
            }
            response = requests.get(url, auth=(SONARCLOUD_TOKEN, ""), params=params)
            response.raise_for_status()
            result = response.json()

            issues = result.get("issues", [])
            total_issues.extend(issues)

            if page * page_size >= result.get("total", 0):
                break
            page += 1

        # Tally issues by type and severity
        counts = {}
        for issue in total_issues:
            issue_type = issue.get("type", "UNKNOWN")
            severity = issue.get("severity", "UNKNOWN")
            key = (issue_type, severity)
            counts[key] = counts.get(key, 0) + 1

        # Prepare rows for this branch
        for (issue_type, severity), count in counts.items():
            rows.append({
                "branch": branch,
                "issue_type": issue_type,
                "severity": severity,
                "count": count
            })

    # Export to CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["branch", "issue_type", "severity", "count"])
        writer.writeheader()
        writer.writerows(rows)

In [33]:

branches = get_all_branches(SONAR_PROJECT_KEY)
print(f"Found {len(branches)} branches: {branches}")


Found 19 branches: ['python_claude_3', 'python_claude_2', 'python_claude_1', 'python_deepseek_3', 'python_deepseek_2', 'python_deepseek_1', 'python_gpt_3', 'python_gpt_2', 'python_gpt_1', 'java_claude_3', 'java_claude_2', 'java_claude_1', 'java_deepseek_3', 'java_deepseek_2', 'java_deepseek_1', 'java_gpt_3', 'java_gpt_2', 'java_gpt_1', 'main']


In [54]:
export_summary_csv(SONAR_PROJECT_KEY, branches)
print(f"Export of summary CSV done!")

Exporting summary metrics for branch: python_claude_3
Exporting summary metrics for branch: python_claude_2
Exporting summary metrics for branch: python_claude_1
Exporting summary metrics for branch: python_deepseek_3
Exporting summary metrics for branch: python_deepseek_2
Exporting summary metrics for branch: python_deepseek_1
Exporting summary metrics for branch: python_gpt_3
Exporting summary metrics for branch: python_gpt_2
Exporting summary metrics for branch: python_gpt_1
Exporting summary metrics for branch: java_claude_3
Exporting summary metrics for branch: java_claude_2
Exporting summary metrics for branch: java_claude_1
Exporting summary metrics for branch: java_deepseek_3
Exporting summary metrics for branch: java_deepseek_2
Exporting summary metrics for branch: java_deepseek_1
Exporting summary metrics for branch: java_gpt_3
Exporting summary metrics for branch: java_gpt_2
Exporting summary metrics for branch: java_gpt_1
Exporting summary metrics for branch: main
Export of

In [35]:
export_issue_severity_breakdown(SONAR_PROJECT_KEY, branches[1:])
print(f"Export of severity breakdown CSV done!")

Gathering severity breakdown for branch: python_claude_2
Gathering severity breakdown for branch: python_claude_1
Gathering severity breakdown for branch: python_deepseek_3
Gathering severity breakdown for branch: python_deepseek_2
Gathering severity breakdown for branch: python_deepseek_1
Gathering severity breakdown for branch: python_gpt_3
Gathering severity breakdown for branch: python_gpt_2
Gathering severity breakdown for branch: python_gpt_1
Gathering severity breakdown for branch: java_claude_3
Gathering severity breakdown for branch: java_claude_2
Gathering severity breakdown for branch: java_claude_1
Gathering severity breakdown for branch: java_deepseek_3
Gathering severity breakdown for branch: java_deepseek_2
Gathering severity breakdown for branch: java_deepseek_1
Gathering severity breakdown for branch: java_gpt_3
Gathering severity breakdown for branch: java_gpt_2
Gathering severity breakdown for branch: java_gpt_1
Gathering severity breakdown for branch: main
Export of

In [36]:
export_detailed_issues(SONAR_PROJECT_KEY, branches[1:])
print(f"Export of detailed issues done!")

Exporting detailed issues for branch: python_claude_2
Exporting detailed issues for branch: python_claude_1
Exporting detailed issues for branch: python_deepseek_3
Exporting detailed issues for branch: python_deepseek_2
Exporting detailed issues for branch: python_deepseek_1
Exporting detailed issues for branch: python_gpt_3
Exporting detailed issues for branch: python_gpt_2
Exporting detailed issues for branch: python_gpt_1
Exporting detailed issues for branch: java_claude_3
Exporting detailed issues for branch: java_claude_2
Exporting detailed issues for branch: java_claude_1
Exporting detailed issues for branch: java_deepseek_3
Exporting detailed issues for branch: java_deepseek_2
Exporting detailed issues for branch: java_deepseek_1
Exporting detailed issues for branch: java_gpt_3
Exporting detailed issues for branch: java_gpt_2
Exporting detailed issues for branch: java_gpt_1
Exporting detailed issues for branch: main
Export of detailed issues done!
